🌻 Import thư viện

In [1]:
from pyspark.sql.types import *
from graphframes import *
from pyspark.sql import SparkSession

In [2]:
import findspark
findspark.init()

In [3]:
spark = SparkSession.builder.appName("BFS-transport").getOrCreate()

22/01/12 21:41:05 WARN Utils: Your hostname, Rose resolves to a loopback address: 127.0.1.1; using 192.168.1.14 instead (on interface enp3s0)
22/01/12 21:41:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/01/12 21:41:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/12 21:41:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/12 21:41:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/01/12 21:41:06 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/01/12 21:41:06 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/01/12 21:41:06 WAR

🌻 Tạo GraphFrames từ file ***.csv**.

In [4]:
def createTransportGraph():
    # custom object
    node_fields = [
        StructField("id", StringType(), True),
        StructField("lattitude", FloatType(), True),
        StructField("longitude", FloatType(), True),
        StructField("population", IntegerType(), True)
    ]
    
    nodes = spark.read.csv("./data/transport-nodes.csv",
                           header=True, schema=StructType(node_fields))
    
    # đọc file thể hiện các mối quan hệ giữa các chuyến bay
    rels = spark.read.csv("./data/transport-relationships.csv", header=True)
    
    reversed_rels = (rels.withColumn("newSrc", rels.dst).
                        withColumn("newDst", rels.src).
                        drop("dst", "src").
                        withColumnRenamed("newSrc", "src").
                        withColumnRenamed("newDst", "dst").
                        select("src", "dst", "relationship", "cost"))
    
    relationships = rels.union(reversed_rels)
    
    return GraphFrame(nodes, relationships)
    

In [5]:
g = createTransportGraph()

In [6]:
g.vertices.filter("population > 100000 and population < 300000").\
    sort("population").show()

+----------+---------+---------+----------+
|        id|lattitude|longitude|population|
+----------+---------+---------+----------+
|Colchester| 51.88921|  0.90421|    104390|
|   Ipswich| 52.05917|  1.15545|    133384|
+----------+---------+---------+----------+



22/01/12 21:41:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, latitude, longitude, population
 Schema: id, lattitude, longitude, population
Expected: lattitude but found: latitude
CSV file: file:///home/manhcuong/Documents/Spark_GraphX_in_Action/chapter_04/data/transport-nodes.csv


In [7]:
from_expr = "id='Den Haag'"
to_expr = "population > 100000 and population < 300000 and id <> 'Den Haag'"
result = g.bfs(from_expr, to_expr)

22/01/12 21:43:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, latitude, longitude, population
 Schema: id, lattitude, longitude, population
Expected: lattitude but found: latitude
CSV file: file:///home/manhcuong/Documents/Spark_GraphX_in_Action/chapter_04/data/transport-nodes.csv
22/01/12 21:43:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, latitude, longitude, population
 Schema: id, lattitude, longitude, population
Expected: lattitude but found: latitude
CSV file: file:///home/manhcuong/Documents/Spark_GraphX_in_Action/chapter_04/data/transport-nodes.csv
22/01/12 21:43:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, latitude, longitude, population
 Schema: id, lattitude, longitude, population
Expected: lattitude but found: latitude
CSV file: file:///home/manhcuong/Documents/Spark_GraphX_in_Action/chapter_04/data/transport-nodes.csv
22/01/12 21:43:52 WARN CSVHeaderChecker: CSV head

In [8]:
print(result.columns)

['from', 'e0', 'v1', 'e1', 'v2', 'e2', 'to']


In [9]:
columns = [column for column in result.columns if not column.startswith("e")]
result.select(columns).show()

22/01/12 21:46:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, latitude, longitude, population
 Schema: id, lattitude, longitude, population
Expected: lattitude but found: latitude
CSV file: file:///home/manhcuong/Documents/Spark_GraphX_in_Action/chapter_04/data/transport-nodes.csv
22/01/12 21:46:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, latitude, longitude, population
 Schema: id, lattitude, longitude, population
Expected: lattitude but found: latitude
CSV file: file:///home/manhcuong/Documents/Spark_GraphX_in_Action/chapter_04/data/transport-nodes.csv
22/01/12 21:46:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, latitude, longitude, population
 Schema: id, lattitude, longitude, population
Expected: lattitude but found: latitude
CSV file: file:///home/manhcuong/Documents/Spark_GraphX_in_Action/chapter_04/data/transport-nodes.csv
22/01/12 21:46:31 WARN CSVHeaderChecker: CSV head

+--------------------+--------------------+--------------------+--------------------+
|                from|                  v1|                  v2|                  to|
+--------------------+--------------------+--------------------+--------------------+
|[Den Haag, 52.078...|[Hoek van Holland...|[Felixstowe, 51.9...|[Ipswich, 52.0591...|
+--------------------+--------------------+--------------------+--------------------+

